In [1]:
import torch
def expand(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'], st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'], st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

def expand_three_times_truncate(st_dict, new_path):
    st_dict['layoutlmv3.embeddings.position_ids'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_ids'],
         st_dict['layoutlmv3.embeddings.position_ids'],
         st_dict['layoutlmv3.embeddings.position_ids'])
        , dim=1)
    st_dict['layoutlmv3.embeddings.position_embeddings.weight'] = torch.cat(
        (st_dict['layoutlmv3.embeddings.position_embeddings.weight'],
         st_dict['layoutlmv3.embeddings.position_embeddings.weight'],
         st_dict['layoutlmv3.embeddings.position_embeddings.weight'])
        , dim=0)
    d = ['x', 'y', 'w', 'h']
    for dd in d: 
        st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'] = torch.cat(
            (st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'], 
             st_dict[f'layoutlmv3.embeddings.{dd}_position_embeddings.weight'])
            , dim=0)
    print("expand position_ids to ", st_dict['layoutlmv3.embeddings.position_ids'].shape)
    print("expand position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.position_embeddings.weight'].shape)
    print("expand x_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.x_position_embeddings.weight'].shape)
    print("expand y_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.y_position_embeddings.weight'].shape)
    print("expand w_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.w_position_embeddings.weight'].shape)
    print("expand h_position_embeddings.weight to ", st_dict['layoutlmv3.embeddings.h_position_embeddings.weight'].shape)
    for k in list(st_dict.keys()):
        if k.startswith('classifier.') or k.endswith('lam'):
            st_dict.pop(k)
            print("pop ", k)
    torch.save(st_dict, new_path)

/root/mambaforge/envs/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForRelationExtraction, LayoutLMv3ForTokenClassification
# model_ft = LayoutLMv3ForTokenClassification.from_pretrained(
#     "HYPJUDY/layoutlmv3-large-finetuned-funsd"
# )

In [4]:
# re_pretrain = LayoutLMv3ForRelationExtraction.from_pretrained(
#     "microsoft/layoutlmv3-large"
# )
ner_pretrain = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-large"
)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-large and are newly initialized: ['layoutlmv3.encoder.layer.11.attention.self.lam', 'layoutlmv3.encoder.layer.9.attention.self.lam', 'layoutlmv3.encoder.layer.20.attention.self.lam', 'layoutlmv3.encoder.layer.23.attention.self.lam', 'layoutlmv3.encoder.layer.22.attention.self.lam', 'layoutlmv3.encoder.layer.21.attention.self.lam', 'layoutlmv3.encoder.layer.0.attention.self.lam', 'layoutlmv3.encoder.layer.7.attention.self.lam', 'layoutlmv3.encoder.layer.13.attention.self.lam', 'classifier.weight', 'layoutlmv3.encoder.layer.14.attention.self.lam', 'layoutlmv3.encoder.layer.4.attention.self.lam', 'layoutlmv3.encoder.layer.6.attention.self.lam', 'layoutlmv3.encoder.layer.5.attention.self.lam', 'layoutlmv3.encoder.layer.17.attention.self.lam', 'layoutlmv3.encoder.layer.15.attention.self.lam', 'layoutlmv3.encoder.layer.3.attention.self.lam', 'layoutlmv3.encoder.layer.18.att

In [ ]:
# re_st_dict = re_pretrain.state_dict()
# expand(re_st_dict, "layoutlmv3-large-re-1028/pytorch_model.bin")

In [14]:
ner_st_dict = ner_pretrain.state_dict()
expand_truncate(ner_st_dict, "layoutlmv3-large-ner-1028/pytorch_model.bin")

expand position_ids to  torch.Size([1, 1028])
expand position_embeddings.weight to  torch.Size([1028, 1024])
pop  layoutlmv3.encoder.layer.0.attention.self.lam
pop  layoutlmv3.encoder.layer.1.attention.self.lam
pop  layoutlmv3.encoder.layer.2.attention.self.lam
pop  layoutlmv3.encoder.layer.3.attention.self.lam
pop  layoutlmv3.encoder.layer.4.attention.self.lam
pop  layoutlmv3.encoder.layer.5.attention.self.lam
pop  layoutlmv3.encoder.layer.6.attention.self.lam
pop  layoutlmv3.encoder.layer.7.attention.self.lam
pop  layoutlmv3.encoder.layer.8.attention.self.lam
pop  layoutlmv3.encoder.layer.9.attention.self.lam
pop  layoutlmv3.encoder.layer.10.attention.self.lam
pop  layoutlmv3.encoder.layer.11.attention.self.lam
pop  layoutlmv3.encoder.layer.12.attention.self.lam
pop  layoutlmv3.encoder.layer.13.attention.self.lam
pop  layoutlmv3.encoder.layer.14.attention.self.lam
pop  layoutlmv3.encoder.layer.15.attention.self.lam
pop  layoutlmv3.encoder.layer.16.attention.self.lam
pop  layoutlmv3.e

In [5]:
ner_st_dict = ner_pretrain.state_dict()
expand_three_times_truncate(ner_st_dict, "layoutlmv3-large-ner-1542/pytorch_model.bin")

expand position_ids to  torch.Size([1, 1542])
expand position_embeddings.weight to  torch.Size([1542, 1024])
expand x_position_embeddings.weight to  torch.Size([2048, 171])
expand y_position_embeddings.weight to  torch.Size([2048, 171])
expand w_position_embeddings.weight to  torch.Size([2048, 170])
expand h_position_embeddings.weight to  torch.Size([2048, 170])
pop  layoutlmv3.encoder.layer.0.attention.self.lam
pop  layoutlmv3.encoder.layer.1.attention.self.lam
pop  layoutlmv3.encoder.layer.2.attention.self.lam
pop  layoutlmv3.encoder.layer.3.attention.self.lam
pop  layoutlmv3.encoder.layer.4.attention.self.lam
pop  layoutlmv3.encoder.layer.5.attention.self.lam
pop  layoutlmv3.encoder.layer.6.attention.self.lam
pop  layoutlmv3.encoder.layer.7.attention.self.lam
pop  layoutlmv3.encoder.layer.8.attention.self.lam
pop  layoutlmv3.encoder.layer.9.attention.self.lam
pop  layoutlmv3.encoder.layer.10.attention.self.lam
pop  layoutlmv3.encoder.layer.11.attention.self.lam
pop  layoutlmv3.encod

In [ ]:
# re_ft_st_dict = model_ft.state_dict()
# expand_truncate(re_ft_st_dict, "layoutlmv3-large-re-1028-ft/pytorch_model.bin")

In [ ]:
# ner_ft_st_dict = model_ft.state_dict()
# expand(ner_ft_st_dict, "layoutlmv3-large-ner-1028-ft/pytorch_model.bin")

In [4]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
from layoutlmft.models.layoutlmv3.modeling_layoutlmv3 import LayoutLMv3ForTokenClassification
large_ner_1028 = LayoutLMv3ForTokenClassification.from_pretrained(
    "layoutlmv3-large-ner-1028"
)
print(count_parameters(large_ner_1028))


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at layoutlmv3-large-ner-1028 and are newly initialized: ['layoutlmv3.encoder.layer.6.attention.self.lam', 'layoutlmv3.encoder.layer.9.attention.self.lam', 'layoutlmv3.encoder.layer.4.attention.self.lam', 'layoutlmv3.encoder.layer.13.attention.self.lam', 'layoutlmv3.encoder.layer.14.attention.self.lam', 'layoutlmv3.encoder.layer.10.attention.self.lam', 'layoutlmv3.encoder.layer.21.attention.self.lam', 'layoutlmv3.encoder.layer.5.attention.self.lam', 'layoutlmv3.encoder.layer.7.attention.self.lam', 'classifier.weight', 'layoutlmv3.encoder.layer.12.attention.self.lam', 'layoutlmv3.encoder.layer.2.attention.self.lam', 'layoutlmv3.encoder.layer.23.attention.self.lam', 'layoutlmv3.encoder.layer.3.attention.self.lam', 'layoutlmv3.encoder.layer.15.attention.self.lam', 'layoutlmv3.encoder.layer.19.attention.self.lam', 'layoutlmv3.encoder.layer.0.attention.self.lam', 'layoutlmv3.encoder.layer.8.attent

356533786


In [ ]:
model_from_path = LayoutLMv3ForTokenClassification.from_pretrained(
    "layoutlmv3-large-ner-1028",
    # num_labels=2
)
